In [1]:
import os
import sys
import numpy as np
import pandas as pd


import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

sys.path.append('../')

from modules.classifier import DenseLayer, BCELoss
from modules.dataloader import load_npy_files
from modules.linear_transformation import LinearTransformations

### Data Loading

In [2]:
class MultimodalDataset(Dataset):
    def __init__(self, id_label_df, text_features, audio_features, video_features):
        self.id_label_df = id_label_df
        
        # Convert feature lists to dictionaries for fast lookup
        self.text_features = {os.path.basename(file).split('.')[0]: tensor for file, tensor in text_features}
        self.audio_features = {os.path.basename(file).split('_')[1].split('.')[0]: tensor for file, tensor in audio_features}
        self.video_features = {os.path.basename(file).split('_')[0]: tensor for file, tensor in video_features}

        # List to store missing files
        self.missing_files = []

        # Filter out entries with missing files
        self.valid_files = self._filter_valid_files()


    def _filter_valid_files(self):
        valid_files = []
        for idx in range(len(self.id_label_df)):
            imdbid = self.id_label_df.iloc[idx]['IMDBid']

            # Check if the IMDBid exists in each modality's features
            if imdbid in self.text_features and imdbid in self.audio_features and imdbid in self.video_features:
                valid_files.append(idx)
            else:
                self.missing_files.append({'IMDBid': imdbid})

        # Print missing files after checking all
        if self.missing_files:
            print("Missing files:")
            for item in self.missing_files:
                print(f"IMDBid: {item['IMDBid']}")
            print(f"Total IMDB IDs with missing files: {len(self.missing_files)}")
        else:
            print("No missing files.")

        return valid_files

    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        # Get the original index from the filtered valid files
        original_idx = self.valid_files[idx]
        imdbid = self.id_label_df.iloc[original_idx]['IMDBid']
        label = self.id_label_df.iloc[original_idx]['Label']

        # Retrieve data from the loaded features
        text_data = self.text_features.get(imdbid, torch.zeros((1024,)))
        audio_data = self.audio_features.get(imdbid, torch.zeros((1, 197, 768)))
        video_data = self.video_features.get(imdbid, torch.zeros((95, 768)))
        
        # Define label mapping
        label_map = {'red': 0, 'green': 1} 
        
        # Convert labels to tensor using label_map
        try:
            label_data = torch.tensor([label_map[label]], dtype=torch.float32)  # Ensure labels are integers
        except KeyError as e:
            print(f"Error: Label '{e}' not found in label_map.")
            raise

        return text_data, audio_data, video_data, label_data


In [3]:
import torch.nn.functional as F

def collate_fn(batch):
    text_data, audio_data, video_data, label_data = zip(*batch)

    # Convert lists to tensors
    text_data = torch.stack(text_data)
    audio_data = torch.stack(audio_data)

    # Padding for video data
    # Determine maximum length of video sequences in the batch
    video_lengths = [v.size(0) for v in video_data]
    max_length = max(video_lengths)

    # Pad video sequences to the maximum length
    video_data_padded = torch.stack([
        F.pad(v, (0, 0, 0, max_length - v.size(0)), "constant", 0)
        for v in video_data
    ])

    # Convert labels to tensor and ensure the shape [batch_size, 1]
    label_data = torch.stack(label_data)  # Convert list of tensors to a single tensor

    return text_data, audio_data, video_data_padded, label_data


In [4]:
# Load the labels DataFrame
id_label_df = pd.read_excel('C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\MM-Trailer_dataset.xlsx')

# Define the directories
text_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\textStream_BERT\\feature_vectors\\feature_vectors'
audio_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\audio_fe\\logmel_spectrograms'
video_features_dir = 'C:\\Users\\edjin\\OneDrive\\Documents\\Programming Files\\Thesis\\SMCA\\misc\\visualStream_ViT\\feature_vectors'

# Load the feature vectors from each directory
text_features = load_npy_files(text_features_dir)
audio_features = load_npy_files(audio_features_dir)
video_features = load_npy_files(video_features_dir)

# Splitting data for training, validation, and testing
train_df, val_test_df = train_test_split(id_label_df, test_size=0.3, random_state=42)

# Further splitting remaining set into validation and test sets
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

# Create datasets
train_dataset = MultimodalDataset(train_df, text_features, audio_features, video_features)
val_dataset = MultimodalDataset(val_df, text_features, audio_features, video_features)
test_dataset = MultimodalDataset(test_df, text_features, audio_features, video_features)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn)

# Combine all data for K-fold cross-validation
full_dataset = MultimodalDataset(id_label_df, text_features, audio_features, video_features)



Missing files:
IMDBid: tt2494280
IMDBid: tt1724962
IMDBid: tt1152836
IMDBid: tt0389790
IMDBid: tt3053228
IMDBid: tt1045778
IMDBid: tt1758795
IMDBid: tt0099385
IMDBid: tt2917484
IMDBid: tt4769836
IMDBid: tt0089652
IMDBid: tt0465494
IMDBid: tt3675748
IMDBid: tt2126362
IMDBid: tt0988083
IMDBid: tt2101341
IMDBid: tt0401997
IMDBid: tt1661461
IMDBid: tt1313139
IMDBid: tt1094661
IMDBid: tt5162658
IMDBid: tt0104839
IMDBid: tt1288558
IMDBid: tt5962210
IMDBid: tt2937696
IMDBid: tt0284363
IMDBid: tt5580390
IMDBid: tt2293750
IMDBid: tt2980472
IMDBid: tt0082186
IMDBid: tt0924129
IMDBid: tt0988595
IMDBid: tt1349482
IMDBid: tt4158096
IMDBid: tt1403241
IMDBid: tt2713642
IMDBid: tt1682940
IMDBid: tt10327354
IMDBid: tt1087842
IMDBid: tt1800302
IMDBid: tt0113855
IMDBid: tt2504022
IMDBid: tt7248248
IMDBid: tt1720164
IMDBid: tt1336621
IMDBid: tt0266987
IMDBid: tt0859635
Total IMDB IDs with missing files: 47
Missing files:
IMDBid: tt2437712
IMDBid: tt0099371
IMDBid: tt2935564
IMDBid: tt0140336
IMDBid: tt468

### Test Dataloader

In [5]:
for text, audio, video, labels in train_dataloader:
    # print(f"Text Shape: {text.shape}")
    # print(f"Audio Shape: {audio.shape}")
    # print(f"Video Shape: {video.shape}")
    # print(f"Labels Shape: {labels.shape}")
    print('---')


---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---


In [6]:
for text_features, audio_features, video_features, targets in train_dataloader:
    print("Text Features Shape:", text_features.shape)
    print("Audio Features Shape:", audio_features.shape)
    print("Video Features Shape:", video_features.shape)
    print("Labels shape:", targets.shape)
    break


Text Features Shape: torch.Size([8, 1024])
Audio Features Shape: torch.Size([8, 1, 197, 768])
Video Features Shape: torch.Size([8, 147, 768])
Labels shape: torch.Size([8, 1])


In [7]:
# Function to print a sample from the dataset
def print_sample(dataset, index):
    text_data, audio_data, video_data, label_data = dataset[index]
    print(f"Sample {index}:")
    # print("Text Data:", text_data)
    # print("Audio Data:", audio_data)
    # print("Video Data:", video_data)
    print("-" * 30)
    print("Text Data Shape:", text_data.shape)
    print("Audio Data Shape:", audio_data.shape)
    print("Video Data Shape:", video_data.shape)
    print("Label:", label_data)
    print("-" * 30)

# Print a sample from each dataset
print("Training Dataset Sample:")
print_sample(train_dataset, 1)  # Change 5 to any index to view different samples

# print("Validation Dataset Sample:")
# print_sample(val_dataset, 0)  # Change 5 to any index to view different samples

# print("Test Dataset Sample:")
# print_sample(test_dataset, 0)  # Change 5 to any index to view different samples

Training Dataset Sample:
Sample 1:
------------------------------
Text Data Shape: torch.Size([1024])
Audio Data Shape: torch.Size([1, 197, 768])
Video Data Shape: torch.Size([89, 768])
Label: tensor([0.])
------------------------------


In [8]:
def print_dataloader_samples(dataloader, num_batches=1):
    for i, batch in enumerate(dataloader):
        if i >= num_batches:
            break
        
        text_data, audio_data, video_data, labels_data = batch

        # # Convert labels to a list of integers if they are tensors
        # if isinstance(labels, torch.Tensor):
        #     labels = labels.tolist()

        print(f"Batch {i}:")
        print("Text Data Shape:", text_data.shape)
        print("Audio Data Shape:", audio_data.shape)
        print("Video Data Shape:", video_data.shape)
        print("Labels:", labels_data.shape)
        print("-" * 30)

# Print a few batches from the training DataLoader
print("Training DataLoader Samples:")
print_dataloader_samples(train_dataloader, num_batches=2)

# # Print a few batches from the validation DataLoader
# print("Validation DataLoader Samples:")
# print_dataloader_samples(val_dataloader, num_batches=5)

# # Print a few batches from the validation DataLoader
# print("Validation DataLoader Samples:")
# print_dataloader_samples(test_dataloader, num_batches=5)

Training DataLoader Samples:
Batch 0:
Text Data Shape: torch.Size([8, 1024])
Audio Data Shape: torch.Size([8, 1, 197, 768])
Video Data Shape: torch.Size([8, 185, 768])
Labels: torch.Size([8, 1])
------------------------------
Batch 1:
Text Data Shape: torch.Size([8, 1024])
Audio Data Shape: torch.Size([8, 1, 197, 768])
Video Data Shape: torch.Size([8, 142, 768])
Labels: torch.Size([8, 1])
------------------------------


### GMU Class

In [9]:
# Class for Gated Multimodal Unit of Arevalo et al. (2017)
class GatedMultimodalUnit(torch.nn.Module):
    def __init__(self, text_dim, audio_dim, video_dim, output_dim):
        super(GatedMultimodalUnit, self).__init__()
        
        # Linear transformation for text
        self.text_linear = LinearTransformations(text_dim, output_dim)
        
        # Convolutional layers for audio and video features
        self.audio_conv = nn.Conv1d(audio_dim, output_dim, kernel_size=1)
        self.video_conv = nn.Conv1d(video_dim, output_dim, kernel_size=1)
        
        self.output_dim = output_dim 
        
        # Activation functions
        self.activation = nn.Tanh()
        self.gate_activation = nn.Sigmoid()
        
        # Weight matrices for each modality
        self.W1 = nn.Parameter(torch.Tensor(output_dim, output_dim))
        self.W2 = nn.Parameter(torch.Tensor(output_dim, output_dim))
        self.W3 = nn.Parameter(torch.Tensor(output_dim, output_dim))
        
        # Gating matrices
        self.Y1 = nn.Parameter(torch.Tensor(output_dim, output_dim * 3))
        self.Y2 = nn.Parameter(torch.Tensor(output_dim, output_dim * 3))
        self.Y3 = nn.Parameter(torch.Tensor(output_dim, output_dim * 3))
        
        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        
        # Initialize weight matrices
        init.xavier_uniform_(self.W1)
        init.xavier_uniform_(self.W2)
        init.xavier_uniform_(self.W3)
        
        # Initialize gating matrices
        init.xavier_uniform_(self.Y1)
        init.xavier_uniform_(self.Y2)
        init.xavier_uniform_(self.Y3)
        
        
    def forward(self, text_features, audio_features, video_features):

        # Process text features
        x_t = self.text_linear(text_features)  # Shape: [batch_size, output_dim]
        h1 = self.activation(torch.matmul(x_t, self.W1))    # Shape: [batch_size, output_dim]

        # Process audio features
        audio_features = audio_features.squeeze(1).permute(0, 2, 1)     # Shape: [batch_size, audio_dim, sequence_length] 
        x_a = self.audio_conv(audio_features).mean(dim=-1)              # Shape: [batch_size, output_dim]
        h2 = self.activation(torch.matmul(x_a, self.W2))                # Shape: [batch_size, output_dim]

        # Process video features
        video_features = video_features.permute(0, 2, 1)   # Shape: [batch_size, video_dim, sequence_length]
        x_v = self.video_conv(video_features).mean(dim=-1)              # Shape: [batch_size, output_dim]
        h3 = self.activation(torch.matmul(x_v, self.W3))                # Shape: [batch_size, output_dim]

        # Combine processed features
        x = torch.cat((x_t, x_a, x_v), dim=1)
        
        # Compute gating weights for each modality
        z1 = self.gate_activation(torch.matmul(x, self.Y1.t()))
        z2 = self.gate_activation(torch.matmul(x, self.Y2.t()))
        z3 = self.gate_activation(torch.matmul(x, self.Y3.t()))

        # Calculate final output
        h = z1 * h1 + z2 * h2 + z3 * h3

        return h


### Test Model

In [10]:
# Test the GMU model using the items from dataloader as input

# Define dimensions
text_dim = 1024
audio_dim = 768  # Number of channels in audio data
video_dim = 768  # Number of channels in video data
output_dim = 512  # You can set this to any value, depending on your requirements

# Instantiate the GMU model
gmu = GatedMultimodalUnit(text_dim, audio_dim, video_dim, output_dim)

# Use DataLoader to get a batch of data
for batch in train_dataloader:  # You can use any DataLoader (train_dataloader, val_dataloader, etc.)
    text_data, audio_data, video_data, labels = batch
    
   
    # Feed the entire batch to the GMU model
    with torch.no_grad():
        output = gmu(text_data, audio_data, video_data)
    
    # Print the output shape
    print('-'*50)
    print("GMU Output Shape:", output.shape)
    print("GMU Output: ", output)
    
    # Break after the first batch for testing purposes
    break


--------------------------------------------------
GMU Output Shape: torch.Size([8, 512])
GMU Output:  tensor([[ 0.2707, -0.3761, -0.0439,  ...,  0.0531,  0.1791,  0.2163],
        [ 0.1478, -0.3379, -0.0695,  ..., -0.1187,  0.1509,  0.1994],
        [ 0.2109, -0.2986, -0.2121,  ...,  0.1373,  0.2170,  0.1805],
        ...,
        [ 0.3014, -0.1418, -0.1360,  ...,  0.1287,  0.2102, -0.0262],
        [ 0.3005, -0.2849, -0.2008,  ...,  0.1474,  0.2481,  0.1312],
        [ 0.2392, -0.3535, -0.1417,  ...,  0.0201,  0.2196,  0.1913]])


### Model Training

In [11]:
import torch.optim as optim

def get_optimizer(parameters, lr=1e-3):
    # Create an optimizer, for example, Adam
    return optim.Adam(parameters, lr=lr)

In [12]:
def train_model(model, dense_layer, dataloader, criterion, optimizer, device):
    model.train()
    dense_layer.train()
    total_loss = 0
    
    for text_features, audio_features, video_features, targets in dataloader:
        text_features, audio_features, video_features, targets = (
            text_features.to(device),
            audio_features.to(device),
            video_features.to(device),
            targets.to(device)
        )
        
        optimizer.zero_grad()
        
        # Pass inputs through GMU model
        outputs = model(text_features, audio_features, video_features)  # Shape: [batch_size, output_dim]
        
        # Pass the GMU outputs through the dense layer to get final predictions
        predictions = dense_layer(outputs)  # Shape: [batch_size, 1]
        
        # Compute loss
        loss = criterion(predictions, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(dataloader)
    print(f"Training Loss: {average_loss:.4f}")
    return average_loss

In [13]:
import torch
from torcheval.metrics import BinaryPrecision, BinaryRecall, BinaryF1Score

def evaluate_model(model, dense_layer, dataloader, criterion, device):
    model.eval()
    dense_layer.eval()
    total_loss = 0

    # Initialize the metrics for binary classification
    precision_metric = BinaryPrecision().to(device)
    recall_metric = BinaryRecall().to(device)
    f1_metric = BinaryF1Score().to(device)
    
    with torch.no_grad():
        for text_features, audio_features, video_features, targets in dataloader:
            text_features, audio_features, video_features, targets = (
                text_features.to(device),
                audio_features.to(device),
                video_features.to(device),
                targets.to(device).squeeze()
            )

            # Pass inputs through GMU model
            outputs = model(text_features, audio_features, video_features)
            
            # Pass the GMU outputs through the dense layer to get final predictions
            predictions = dense_layer(outputs).squeeze()  
            
            # Compute loss
            loss = criterion(predictions, targets)
            total_loss += loss.item()

            # Apply threshold to get binary predictions
            preds = (predictions > 0.5).float()
            
            # Convert predictions and targets to integers (for bitwise operations)
            preds_int = preds.long()
            targets_int = targets.long()

            # Update the precision, recall, and F1 score metrics
            precision_metric.update(preds_int, targets_int)
            recall_metric.update(preds_int, targets_int)
            f1_metric.update(preds_int, targets_int)

    # Compute precision, recall, and F1 score
    precision = precision_metric.compute().item()
    recall = recall_metric.compute().item()
    f1_score = f1_metric.compute().item()

    average_loss = total_loss / len(dataloader)

    print(f"Evaluation Loss: {average_loss:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    
    return average_loss, precision, recall, f1_score

In [14]:
import torch
from torch.utils.data import Subset
from sklearn.model_selection import KFold

def cross_validate_model(text_dim, audio_dim, video_dim, output_dim, model_class,  dense_layer_class, dataset, criterion, optimizer_class, device, n_splits, collate_fn):
    kf = KFold(n_splits=n_splits, shuffle=True)
    
    total_loss = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    for fold, (train_index, val_index) in enumerate(kf.split(dataset), 1):
        print(f"Fold {fold}/{n_splits}")

        # Create subsets for training and validation
        train_subset = Subset(dataset, train_index)
        val_subset = Subset(dataset, val_index)
        
        # DataLoaders with batch size 8 and collate function
        train_loader = DataLoader(train_subset, batch_size=8, shuffle=True, num_workers=0, collate_fn=collate_fn)
        val_loader = DataLoader(val_subset, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn)
        
        # Initialize the model and dense layer for the current fold
        model = model_class(text_dim=text_dim, audio_dim=audio_dim, video_dim=video_dim, output_dim=output_dim).to(device)
        dense_layer = dense_layer_class(input_size=output_dim).to(device)
        
        # Combine parameters of GMU model and DenseLayer for the optimizer
        optimizer = optimizer_class(list(model.parameters()) + list(dense_layer.parameters()))
        
        print(f"Training model for fold {fold}")
        train_loss = train_model(model, dense_layer, train_loader, criterion, optimizer, device)
        
        print(f"Evaluating model for fold {fold}")
        val_loss, precision, recall, f1_score = evaluate_model(model, dense_layer, val_loader, criterion, device)
        
        total_loss += val_loss
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score
    
    average_cv_loss = total_loss / n_splits
    average_cv_precision = total_precision / n_splits
    average_cv_recall = total_recall / n_splits
    average_cv_f1 = total_f1 / n_splits
    
    print(f"Average Cross-Validation Loss: {average_cv_loss:.4f}")
    print(f"Average Cross-Validation Precision: {average_cv_precision:.4f}")
    print(f"Average Cross-Validation Recall: {average_cv_recall:.4f}")
    print(f"Average Cross-Validation F1 Score: {average_cv_f1:.4f}")
    
    return average_cv_loss, average_cv_precision, average_cv_recall, average_cv_f1


### Run

In [17]:
import torch
from torch.utils.data import DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dimensions (example values)
text_dim = 1024  # Example text feature dimension
audio_dim = 768   # Example audio feature dimension (e.g., after pooling or processing)
video_dim = 768   # Example video feature dimension (e.g., after pooling or processing)
output_dim = 512  # Example output dimension of the GMU before the dense layer

# Cross-validation
average_cv_loss = cross_validate_model(
    text_dim=text_dim,
    audio_dim=audio_dim,
    video_dim=video_dim,
    output_dim=output_dim,
    model_class=GatedMultimodalUnit,
    dense_layer_class=DenseLayer,
    dataset=full_dataset,  # Use your complete dataset for cross-validation
    criterion=BCELoss(),
    optimizer_class=get_optimizer,  # Pass optimizer class, not the instantiated optimizer
    device=device,
    n_splits=10,
    collate_fn=collate_fn
)


Fold 1/10
Training model for fold 1
Training Loss: 0.4334
Evaluating model for fold 1
Evaluation Loss: 0.8081
Precision: 0.9815
Recall: 0.5196
F1 Score: 0.6795
Fold 2/10
Training model for fold 2
Training Loss: 0.4776
Evaluating model for fold 2
Evaluation Loss: 0.3081
Precision: 0.9500
Recall: 0.8879
F1 Score: 0.9179
Fold 3/10
Training model for fold 3
Training Loss: 0.4532
Evaluating model for fold 3
Evaluation Loss: 0.3941
Precision: 0.8198
Recall: 0.9891
F1 Score: 0.8966
Fold 4/10
Training model for fold 4
Training Loss: 0.4509
Evaluating model for fold 4
Evaluation Loss: 0.2693
Precision: 0.8966
Recall: 0.9811
F1 Score: 0.9369
Fold 5/10
Training model for fold 5
Training Loss: 0.4591
Evaluating model for fold 5
Evaluation Loss: 0.5946
Precision: 0.7778
Recall: 1.0000
F1 Score: 0.8750
Fold 6/10
Training model for fold 6
Training Loss: 0.4762
Evaluating model for fold 6
Evaluation Loss: 0.3828
Precision: 0.8762
Recall: 0.9485
F1 Score: 0.9109
Fold 7/10
Training model for fold 7
Trai

In [16]:
# # Device configuration
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Define the dimensions (example values)
# text_dim = 1024  # Example text feature dimension
# audio_dim = 768   # Example audio feature dimension (e.g., after pooling or processing)
# video_dim = 768   # Example video feature dimension (e.g., after pooling or processing)
# output_dim = 512  # Example output dimension of the GMU before the dense layer

# # Initialize the model and dense layer with these dimensions
# model = GatedMultimodalUnit(text_dim=text_dim, audio_dim=audio_dim, video_dim=video_dim, output_dim=output_dim).to(device)
# dense_layer = DenseLayer(input_size=output_dim).to(device)  # Make sure 512 matches the output size of GatedMultimodalUnit
# criterion = BCELoss()  # Use BCELoss for binary classification

# # Create DataLoaders
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0, collate_fn=collate_fn)
# val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn)
# test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn)

# # Set up optimizer
# optimizer = get_optimizer(list(model.parameters()) + list(dense_layer.parameters()))

# # # Train the model
# # train_loss = train_model(model, dense_layer, train_dataloader, criterion, optimizer, device)

# # # Evaluate the model (use val_dataloader for validation evaluation)
# # eval_loss = evaluate_model(model, dense_layer, val_dataloader, criterion, device)

# # Cross-validation
# average_cv_loss = cross_validate_model(
#     model_class=GatedMultimodalUnit,
#     dense_layer_class=DenseLayer,
#     dataset=full_dataset,  # Use your complete dataset for cross-validation
#     collate_fn=collate_fn
#     criterion=criterion,
#     optimizer_class=get_optimizer,
#     device=device,
#     n_splits=5
# )

# print(f"Training Loss: {train_loss:.4f}")
# print(f"Validation Loss: {eval_loss:.4f}")
# print(f"Average Cross-Validation Loss: {average_cv_loss:.4f}")
